In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import importlib

import itertools as it
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import statsmodels.api as sm

from itertools import product
from pandas_profiling import ProfileReport

from criteriaetl.utils.expansion_func import (get_value_counts_with_expansion_factor,
    get_percentage_table_with_expansion_factor)
from criteriaetl.utils.display_func import cdisplay, percentage_count_plot
from criteriaetl.utils.common_func import (get_weighted_complete_randomization_series_on_subset, 
    proportional_cut, weighted_qcut, get_partition_bool_columns_dict)
from criteriaetl.transformers.columns_base import (NameTransformer, 
    ReplaceTransformer, SelectTransformer, AssignTransformer)
from criteriaetl.transformers.rows_base import AggregateTransformer
from criteriaetl.transformers.fusion_base import MergeTransformer

from projectetl.utils.dataload import (load_survey_data_do, save_survey_with_pickle,
                                       load_survey_from_pickle)
from projectetl.utils.config import ENHOGAR_DIR

In [3]:
survey_elected_raw = load_survey_data_do(ENHOGAR_DIR / 'Elegidos_ENHOGAR2016_PUB.sav',
                                         load_func=lambda path: pd.read_spss(str(path)))
survey_member_raw = load_survey_data_do(ENHOGAR_DIR / 'Personas_ENHOGAR2016_PUB.sav',
                                        load_func=lambda path: pd.read_spss(str(path)))
survey_house_raw = load_survey_data_do(ENHOGAR_DIR / 'Viviendas_ENHOGAR2016_PUB.sav',
                                       load_func=lambda path: pd.read_spss(str(path)))

In [4]:
survey_member_raw.iloc[:100].to_csv('ENHOGAR (sample2).csv')

In [35]:
# hhogar 
# ---
# h402 = sexo
# h403 = edad
# h405 = parentesco
# h407 = estado civil (h407a = números)
# ----
# h409 = aprendió a leer y escribir
# h410 = donde aprendió a leer y escribir
# h411 = asiste o asistió a centro educativo
# h412 = nivel educativo más alto
# h413 = curso o año más alto que cursó
# h414 = asiste en el actual año escolar a algún centro educativo
# h415 = nivel educativo al que está asistiendo
# h416 = curso o año al que está asistiendo
# ---
# h501 = ¿Trabajó o realizó (nombre) alguna actividad económica por lo menos una hora la semana pasada?
# h502 = Aunque no trabajó la semana pasada ¿Tenía algún empleo, negocio o actividad?
# h503 = ¿La semana pasada (nombre) cultivó, cosechó o cuidó ganado u otros animales?
# h504 = ¿La semana pasada, elaboró (nombre) algún producto como artesanía o comida para vender?
# h505 = ¿La semana pasada, ayudó (nombre) a algún familiar en su negocio, empresa o finca?
# h506 = ¿La semana pasada (nombre) cosió, planchó, limpió casas, lavó ropa ajena o realizó otra actividad por paga?
# h507 = Durante las últimas cuatro semanas ¿Ha buscado (nombre) trabajo o estuvo tratando de establecer su propio negocio, actividad económica o empresa?
# h508 = ¿Por qué (nombre) no ha buscado trabajo?
# lh509 = ¿Podría (nombre) haber aceptado un trabajo la semana pasada si le hubieran ofrecido uno?
# lh510 = ¿Dispone (nombre) del tiempo y las condiciones necesarias para salir a trabajar?
# lh511 = La semana pasada, ¿Habría tenido (nombre) el tiempo y las condiciones necesarias para salir a trabajar?
# lh512 = ¿Ha trabajado (nombre) antes por paga o ganancia?
# lh513 = ¿Cuál fue la principal ocupación u oficio línea que desempeñó (nombre) durante la semana pasada o en el último trabajo que tuvo?
# lh514 = ¿A qué se dedica la empresa, negocio o institución en la que trabaja (nombre) actualmente o trabajó por última vez?
# lh515 = ¿En ese trabajo (nombre) es o era...
# lh516 = Regularmente, ¿Cuántas horas a la semana trabaja o trabajó (nombre) en su ocupación principal?
# h601 = ¿Tiene (nombre) PREG. algún seguro de salud, seguro médico o plan de salud?
# region
member_columns = [
    # Identifiers
    'hhogar', 'hzona', 'hprovi', 'h402', 'h403', 'h405', 'h407',
    # Education
    'h409', 'h410', 'h411', 'h412', 'h413', 'h414',
    'h415', 'h416',
    # Economic characteristics
    'h501', 'h502', 'h503', 'h504', 'h505', 'h506', 
    'h507', 'h508', 'lh509', 'lh510', 'lh511', 'lh512', 'lh513',
    'lh514', 'lh515', 'lh516',
    # Assurance
    'h601',
    # Stratos
    'hestrat', 'region', 'grupsec', 'zona', 'hmiembro',
    'hprovin', 'estratos',
    # Factors
    'fexpansion', 'fponderacion'
]

# h702a = Número de personas estándar mayores de 10 en el hogar
# h704 = edad
# h1001 = ¿Trabajó usted la semana estándar pasada por paga o ganacia?
# h1002 = ¿Cuántos días trabajó usted estándar durante la semana pasada?
# h1003 = ¿Cuántas horas trabajó estándar usted diariamente la semana pasada?
# h1005 = ¿Estudia usted estándar actualmente?
# h1009 = ¿Realiza usted algún tipo de estudio vocacional, técnico o profesional como música, pintura, computación u otra actividad similar?
# h1015 = Durante la semana pasada ¿Dedicó usted algún tiempo al cuidado de enfermos, ancianos, niños u otras personas, ya sean de su hogar u otro hogar?
# h1016 = Durante la semana pasada ¿Cuántos días le dedico al cuidado de esa o esas personas?
# h1017 = Durante la semana pasada ¿Cuántas horas le dedicó usted al cuidado de esa o esas personas diariamente o el último día que cuidó a esa o esas personas?
# h1030 = En la semana pasada, ¿Cuántos días dedicó usted tiempo para criar o cuidar animales o realizar algún cultivo o cosecha de productos para el consumo del hogar?
# h1032 = Durante la semana pasada, ¿Cuántos días dedicó usted a realizar algún trabajo sin paga en un negocio o establecimiento de comercio o producción?
# h1034 = En la semana pasada, ¿Cuántos días dedicó usted tiempo para realizar actividades domésticas en su hogar, como limpiar la casa, cocinar, lavar, fregar u otras labores domésticas?
# h1035 = En la semana pasada, ¿Cuántas horas dedicó usted a realizar actividades diariamente o el último día que las realizó la semana pasada?
# h1036 = En la semana pasada, ¿Cuántos días dedicó usted tiempo para realizar actividades domésticas sin paga en otro hogar, como limpiar la casa, cocinar, lavar, fregar u otras labores domésticas?
# h1037 = Durante la semana pasada, ¿Cuántas horas dedicó usted a realizar esas actividades diariamente o el último día que las ralizó la semana pasada?
elected_columns = [
    'hhogar', 'hzona', 'hprovin', 'h703', 'h702a',
    'h704', 'h1001', 'h1002', 'h1003', 'h1005',
    'h1009', 'h1030', 'h1032', 'region', 'hestrat',
    'grupsec', 'fexpansion_tic', 'fponderacion_tic',
    # Target
    'h1015', 'h1016', 'h1017', 'h1034', 'h1035',
    'h1036', 'h1037'
]

# Processing surveys

In [7]:
basic_cols = [
    'hhupm', 'hhviv', 'hhogar'
]

## Assign Transformer

In [19]:
assign_member_map = {
    'temp': lambda _: 1,
    'h703': lambda df, cols=basic_cols: df.groupby(cols)['temp'].transform('cumsum')
}

assign_member_transformer = AssignTransformer(assign_member_map)
survey_member_assigned = assign_member_transformer.transform(survey_member_raw)

In [32]:
survey_member_assigned.head()

,hhupm,hhviv,hhogar,zona,hprovi,hmiembro,h402,h403,h404d,h404m,...,h632,hzona,hprovin,estratos,region,grupsec,fexpansion,fponderacion,temp,h703
0,1.0,1.0,1.0,Urbano,Distrito Nacional,6.0,Hombre,41.0,13.0,12.0,...,No,Urbano,Distrito Nacional,Ciudad de Santo Domingo,Metropolitana,Medio bajo,143.43477,1.080379,1,1
1,1.0,1.0,1.0,Urbano,Distrito Nacional,6.0,Mujer,33.0,29.0,9.0,...,No,Urbano,Distrito Nacional,Ciudad de Santo Domingo,Metropolitana,Medio bajo,143.43477,1.080379,1,2
2,1.0,1.0,1.0,Urbano,Distrito Nacional,6.0,Mujer,18.0,11.0,3.0,...,No,Urbano,Distrito Nacional,Ciudad de Santo Domingo,Metropolitana,Medio bajo,143.43477,1.080379,1,3
3,1.0,1.0,1.0,Urbano,Distrito Nacional,6.0,Hombre,14.0,23.0,12.0,...,No,Urbano,Distrito Nacional,Ciudad de Santo Domingo,Metropolitana,Medio bajo,143.43477,1.080379,1,4
4,1.0,1.0,1.0,Urbano,Distrito Nacional,6.0,Hombre,11.0,13.0,9.0,...,No,Urbano,Distrito Nacional,Ciudad de Santo Domingo,Metropolitana,Medio bajo,143.43477,1.080379,1,5


## Merge Transformer

In [30]:
merge_member_elected_transformer = MergeTransformer(lambda : survey_member_assigned,
                                                    merge_kwargs={
                                                        'on': basic_cols + ['h703'],
                                                        'suffixes': ('', '_other')
                                                    })
survey_elected_merged = merge_member_elected_transformer.transform(survey_elected_raw)

In [31]:
survey_elected_merged

,hhupm,hhviv,hhogar,hzona,hprovin,hresul,hmiembro,helegido,h702a,h702b,...,h631,h632,hzona_other,hprovin_other,estratos,region_other,grupsec_other,fexpansion,fponderacion,temp
0,1.0,1.0,1.0,Urbano,Distrito Nacional,Entrevista completa,6.0,Entrevista completa,5.0,3.0,...,algunas veces?,No,Urbano,Distrito Nacional,Ciudad de Santo Domingo,Metropolitana,Medio bajo,143.43477,1.080379,1
1,1.0,2.0,1.0,Urbano,Distrito Nacional,Entrevista completa,3.0,Entrevista completa,2.0,4.0,...,NaN,NaN,Urbano,Distrito Nacional,Ciudad de Santo Domingo,Metropolitana,Muy bajo,143.43477,1.080379,1
2,1.0,3.0,1.0,Urbano,Distrito Nacional,Entrevista completa,4.0,Entrevista completa,4.0,5.0,...,NaN,NaN,Urbano,Distrito Nacional,Ciudad de Santo Domingo,Metropolitana,Medio y Medio alto,143.43477,1.080379,1
3,1.0,4.0,1.0,Urbano,Distrito Nacional,Entrevista completa,3.0,Entrevista completa,3.0,6.0,...,siempre?,Sí,Urbano,Distrito Nacional,Ciudad de Santo Domingo,Metropolitana,Medio y Medio alto,143.43477,1.080379,1
4,1.0,5.0,1.0,Urbano,Distrito Nacional,Entrevista completa,5.0,Entrevista completa,4.0,7.0,...,NaN,NaN,Urbano,Distrito Nacional,Ciudad de Santo Domingo,Metropolitana,Medio bajo,143.43477,1.080379,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23256,1044.0,20.0,1.0,Rural,El Seibo,Entrevista completa,6.0,Entrevista completa,4.0,2.0,...,NaN,NaN,Rural,El Seibo,Rural,Del Yuma,Bajo,293.70175,2.212220,1
23257,1044.0,21.0,1.0,Rural,El Seibo,Entrevista completa,3.0,Entrevista completa,1.0,3.0,...,siempre?,No,Rural,El Seibo,Rural,Del Yuma,Muy bajo,293.70175,2.212220,1
23258,1044.0,22.0,1.0,Rural,El Seibo,Entrevista completa,3.0,Entrevista completa,3.0,4.0,...,NaN,NaN,Rural,El Seibo,Rural,Del Yuma,Bajo,293.70175,2.212220,1
23259,1044.0,23.0,1.0,Rural,El Seibo,Entrevista completa,2.0,Entrevista completa,2.0,5.0,...,NaN,NaN,Rural,El Seibo,Rural,Del Yuma,Muy bajo,293.70175,2.212220,1


## Final dataframe

In [37]:
survey_elected_final = survey_elected_merged[member_columns + elected_columns]
survey_elected_final

,hhogar,hzona,hprovi,h402,h403,h405,h407,h409,h410,h411,...,grupsec,fexpansion_tic,fponderacion_tic,h1015,h1016,h1017,h1034,h1035,h1036,h1037
0,1.0,Urbano,Distrito Nacional,Hombre,14.0,Adoptado/ hijo de crianza,Soltero(a),Sí,Escuela o colegio,Sí asiste,...,Medio bajo,346.88577,0.965224,No,NaN,NaN,7.0,1.0,97,NaN
1,1.0,Urbano,Distrito Nacional,Hombre,24.0,El (la) Jefe(a),Unido(a),No,NaN,"No asiste, pero asistió",...,Muy bajo,346.88577,0.965224,No,NaN,NaN,97,NaN,97,NaN
2,1.0,Urbano,Distrito Nacional,Mujer,40.0,Hijo(a),Separado(a) de unión libre,Sí,Escuela o colegio,"No asiste, pero asistió",...,Medio y Medio alto,346.88577,0.965224,No,NaN,NaN,4.0,4.0,97,NaN
3,1.0,Urbano,Distrito Nacional,Mujer,86.0,El (la) Jefe(a),Viudo(a),No,NaN,"No asiste, pero asistió",...,Medio y Medio alto,346.88577,0.965224,No,NaN,NaN,7.0,2.0,97,NaN
4,1.0,Urbano,Distrito Nacional,Mujer,17.0,Hijo(a),Soltero(a),Sí,Escuela o colegio,"No asiste, pero asistió",...,Medio bajo,346.88577,0.965224,Si,7.0,3.0,7.0,1.0,97,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23256,1.0,Rural,El Seibo,Mujer,18.0,Hijo(a),Separado(a) de unión libre,Sí,Escuela o colegio,"No asiste, pero asistió",...,Bajo,317.18552,0.882582,No,NaN,NaN,97,NaN,97,NaN
23257,1.0,Rural,El Seibo,Mujer,25.0,El (la) Jefe(a),Separado(a) de unión libre,Sí,Escuela o colegio,"No asiste, pero asistió",...,Muy bajo,317.18552,0.882582,Si,7.0,4.0,4.0,1.0,97,NaN
23258,1.0,Rural,El Seibo,Mujer,72.0,El (la) Jefe(a),Unido(a),No,NaN,9.0,...,Bajo,317.18552,0.882582,No,NaN,NaN,7.0,2.0,97,NaN
23259,1.0,Rural,El Seibo,Mujer,16.0,Esposo(a) o compañero(a),Unido(a),Sí,Escuela o colegio,"No asiste, pero asistió",...,Muy bajo,317.18552,0.882582,Si,7.0,3.0,5.0,1.0,3.0,1.0


# EDA

In [39]:
report = ProfileReport(survey_elected_final, minimal=True)

/Users/ohch/opt/anaconda3/envs/criteria-do/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [40]:
report.to_notebook_iframe()

Render HTML: 100%|██████████| 1/1 [00:00<00:00,  1.05it/s]


In [41]:
report.to_file('report.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 76.21it/s]
